In [19]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [20]:
pd.set_option("display.max_columns", 100)

In [21]:
analysis_date = dt.date(2022,5,25)

In [22]:
itp_id = 300

In [23]:
#route = 3479

In [38]:
(tbl.views.gtfs_schedule_fact_daily_trips()
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
)

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-3768729868815070839,883648,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,53040,54120,0.300000
1,4100255323842349957,-968397926059744982,882520,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2099-01-01,14,14,51840,53160,0.366667
2,4100255323842349957,-5460518217176561126,882433,3489,300,0,2,2022-05-25,0,2022-03-27,2022-08-13,None,None,False,2022-03-29,2099-01-01,14,14,46980,48240,0.350000
3,4100255323842349957,-2259348929513193302,882421,3489,300,0,2,2022-05-25,0,2022-03-27,2022-08-13,None,None,False,2022-03-29,2099-01-01,14,14,63360,64620,0.350000
4,4100255323842349957,-7391811447841817502,883658,3501,300,0,77710,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2099-01-01,14,14,61140,62340,0.333333


In [ ]:
# (tbl.views.gtfs_schedule_fact_daily_trips()
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.service_date == analysis_date)
# )

In [34]:
# (
#     tbl.views.gtfs_rt_fact_daily_feeds()
#     >> filter(_.calitp_itp_id == itp_id)
#     >> arrange(_.date ==analysis_date)
# )

In [40]:
# airtable_organizations = (
#     tbl.airtable.california_transit_organizations()
#     >> select(_.itp_id, _.name, _.caltrans_district,
#               _.website, _.ntp_id, _.drmt_organization_name)
#     >> filter(_.itp_id == itp_id)
#     >> collect()
# )

In [41]:
# airtable_organizations